In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import tensorflow as tf


In [35]:
# Read in CSV file
regional_df = pd.read_csv("Table_CSVs/regional.csv")
regional_df

,DOEID,REGIONC,DIVISION,state_postal,BA_climate,TOTALBTU,TOTALDOL
0,100001,WEST,Mountain South,NM,Mixed-Dry,144647.71,2656.89
1,100002,SOUTH,West South Central,AR,Mixed-Humid,28034.61,975.00
2,100003,WEST,Mountain South,NM,Mixed-Dry,30749.71,522.65
3,100004,SOUTH,South Atlantic,SC,Mixed-Humid,86765.19,2061.77
4,100005,NORTHEAST,Middle Atlantic,NJ,Mixed-Humid,59126.93,1463.04
...,...,...,...,...,...,...,...
18491,118492,SOUTH,South Atlantic,MD,Mixed-Humid,49930.49,1098.51
18492,118493,NORTHEAST,New England,ME,Very-Cold,222186.04,3613.44
18493,118494,SOUTH,West South Central,TX,Hot-Humid,51593.72,1428.31
18494,118495,SOUTH,South Atlantic,SC,Hot-Humid,63555.21,2224.94


In [36]:
# Create DataFrame with regional information
regional_df = regional_df.drop(columns=['DOEID', 'TOTALDOL'])
regional_df.head()

,REGIONC,DIVISION,state_postal,BA_climate,TOTALBTU
0,WEST,Mountain South,NM,Mixed-Dry,144647.71
1,SOUTH,West South Central,AR,Mixed-Humid,28034.61
2,WEST,Mountain South,NM,Mixed-Dry,30749.71
3,SOUTH,South Atlantic,SC,Mixed-Humid,86765.19
4,NORTHEAST,Middle Atlantic,NJ,Mixed-Humid,59126.93


In [37]:
#Checking nulls
regional_df.isnull().sum()

REGIONC         0
DIVISION        0
state_postal    0
BA_climate      0
TOTALBTU        0
dtype: int64

In [39]:
#Separate the features `X` from the target `y`
y = regional_df['TOTALBTU']
X = regional_df.drop(columns='TOTALBTU')

In [40]:
# Encode the categorical variables using get_dummies
X = pd.get_dummies(X)

# Review the features data
X.head()

,REGIONC_MIDWEST,REGIONC_NORTHEAST,REGIONC_SOUTH,REGIONC_WEST,DIVISION_East North Central,DIVISION_East South Central,DIVISION_Middle Atlantic,DIVISION_Mountain North,DIVISION_Mountain South,DIVISION_New England,...,state_postal_WV,state_postal_WY,BA_climate_Cold,BA_climate_Hot-Dry,BA_climate_Hot-Humid,BA_climate_Marine,BA_climate_Mixed-Dry,BA_climate_Mixed-Humid,BA_climate_Subarctic,BA_climate_Very-Cold
0,False,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,False
3,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,False,True,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [41]:
# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,  random_state=42)

In [42]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [43]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=len(X_train_scaled[0]), activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                5920      
                                                                 
 dense_10 (Dense)            (None, 30)                2430      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 8381 (32.74 KB)
Trainable params: 8381 (32.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [45]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100


463/463 [==============================] - 1s 961us/step - loss: -98215360.0000 - accuracy: 0.0000e+00
Epoch 2/100
463/463 [==============================] - 0s 848us/step - loss: -1460597248.0000 - accuracy: 0.0000e+00
Epoch 3/100
463/463 [==============================] - 0s 829us/step - loss: -5994088960.0000 - accuracy: 0.0000e+00
Epoch 4/100
463/463 [==============================] - 0s 866us/step - loss: -15026457600.0000 - accuracy: 0.0000e+00
Epoch 5/100
463/463 [==============================] - 0s 838us/step - loss: -29414764544.0000 - accuracy: 0.0000e+00
Epoch 6/100
463/463 [==============================] - 0s 825us/step - loss: -49698885632.0000 - accuracy: 0.0000e+00
Epoch 7/100
463/463 [==============================] - 0s 946us/step - loss: -76360286208.0000 - accuracy: 0.0000e+00
Epoch 8/100
463/463 [==============================] - 0s 957us/step - loss: -109644824576.0000 - accuracy: 0.0000e+00
Epoch 9/100
463/463 [==============================] - 0s 

In [46]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

116/116 - 0s - loss: -7.7279e+13 - accuracy: 0.0000e+00 - 164ms/epoch - 1ms/step
Loss: -77278516084736.0, Accuracy: 0.0
